In [0]:
# Mounting Google Drive 
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
%cd gdrive/My\ Drive/imagenet

In [0]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import numpy as np
import cv2

In [0]:
page = requests.get('http://image-net.org/challenges/LSVRC/2012/browse-synsets')
soup = BeautifulSoup(page.content, 'html.parser')

In [0]:
# Collecting synset ids 
wnids = []
for link in soup.find_all('a'):
  if 'wnid' in link.get('href'):
    url = urllib.parse.urlparse(link.get('href'))
    params = urllib.parse.parse_qs(url.query)
    if 'wnid' in params:
      wnids.append([params['wnid'][0], link.get_text()])
print(len(wnids))

In [0]:
# For checking if download is being resumed when the notebook is run again
import os
resume = False
if os.path.isdir("./train") == True and os.path.isdir("./validation") == True:
  resume = True
else:
  !mkdir ./train 
  !mkdir ./validation

print(resume)

In [0]:
# Finding out which synset/directory to resume downloading from
resume_index = None

if resume == True:
  last_folder_index = !ls -1t ./train/ | wc -l
  last_folder_index = int(last_folder_index[0])
  
  last_folder_name = wnids[last_folder_index+1][1].split(',')[0].replace(' ', '_').replace("'", "")
  check = !ls -1t ./train/$last_folder_name | wc -l
  print(last_folder_name)
  if int(check[0]) == 101:
    resume_index = last_folder_index + 2
  else:
    !rm -rf ./train/$last_folder_name
    !mkdir ./train/$last_folder_name
    resume_index = last_folder_index

  check = !ls -1t ./train/$last_folder_name | wc -l
  print(check[0])
  
print(wnids[resume_index])
print(resume_index)

In [0]:
def url_to_image(url):
  '''
  download image, convert to NumPy array and return in OpenCV format
  '''
  try:
    resp = urllib.request.urlopen(url, timeout=20)
  except Exception as e:
    return
  
  try:
    resp.getcode()
    image = np.asarray(bytearray(resp.read()), dtype='uint8')
  except Exception as e:
    print(str(e))
    return
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
  return image

In [0]:
def download_images(urls, label):
  count = 0

  for i, url in enumerate(urls):
    try:
      I = url_to_image(url)
    except Exception as e:
      print(str(e))
    
    if I is not None:
      save_path = './train/' + label + '/img' + str(i) + '.jpg'
      print(save_path)
      result = cv2.imwrite(save_path, I)
      if not result:
        print('Could not write image.')
      count += 1
      if count > 100:
        return

In [0]:
# Images downloaded would initially be present in train folder
template = "http://image-net.org/api/text/imagenet.synset.geturls?wnid="
train_dir = "./train/"
validation_dir = "./validation/"

for uid, label in wnids[resume_index:]:
  page_url = template+str(uid)
  print(page_url, label)
  page = requests.get(page_url)
  soup = BeautifulSoup(page.content, 'html.parser')
  str_soup = str(soup)
  split_urls = str_soup.split('\r\n')

  l = label.split(',')[0].replace(' ', '_').replace("'", "")
  t_dir = train_dir + l
  v_dir = validation_dir + l
  print(t_dir)
  !mkdir {t_dir}
  !mkdir {v_dir}

  download_images(split_urls, l)
  